# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Разделение-датасета-на-выборки" data-toc-modified-id="Разделение-датасета-на-выборки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Разделение датасета на выборки</a></span></li><li><span><a href="#Обучение-модели" data-toc-modified-id="Обучение-модели-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обучение модели</a></span></li><li><span><a href="#Среднее-значение-предсказанного-сырья-и-RMSE" data-toc-modified-id="Среднее-значение-предсказанного-сырья-и-RMSE-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Среднее значение предсказанного сырья и RMSE</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Расчёт-прибыли" data-toc-modified-id="Расчёт-прибыли-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Расчёт прибыли</a></span></li><li><span><a href="#Расчёт-рисков" data-toc-modified-id="Расчёт-рисков-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Расчёт рисков</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy import stats as st
from scipy import stats as t

Загрузим данные из 3 регионов и посмотрим на таблицы

In [2]:
df1 = pd.read_csv('/datasets/geo_data_0.csv')
df1
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [3]:
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
df3 = pd.read_csv('/datasets/geo_data_2.csv')
df3.sample(5)

,id,f0,f1,f2,product
54099,kZZcS,-0.928131,1.745928,1.852076,76.577655
64384,QyUlr,1.449800,0.586734,-1.586481,136.898973
89875,OYzhY,-0.687797,0.066099,2.484086,56.886069
11894,t1Z2P,-2.081308,1.177517,-0.282183,105.026948
23965,DKKb1,-1.709942,-2.502277,0.327630,63.739212


Подготовим данные с признаками и целевым признаком для всех таблиц.

In [5]:
target1 = df1['product']
features1 = df1.drop(['id', 'product'], axis=1)

In [6]:
target2 = df2['product']
features2 = df2.drop(['id', 'product'], axis=1)

In [7]:
target3 = df3['product']
features3 = df3.drop(['id', 'product'], axis=1)

## Обучение и проверка модели

### Разделение датасета на выборки
Применим к нашим датасетам функцию train_test_split(), чтоб разделить их на валидационную выборку (25%) и обучающую (75%).

In [8]:
features_train1, features_valid1, target_train1, target_valid1 = train_test_split(features1, target1, 
                                                    test_size=0.25, 
                                                    random_state=12345)

In [9]:
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(features2, target2, 
                                                    test_size=0.25, 
                                                    random_state=12345)

In [10]:
features_train3, features_valid3, target_train3, target_valid3 = train_test_split(features3, target3, 
                                                    test_size=0.25, 
                                                    random_state=12345)

### Обучение модели
Обучим модели линейной регресии на обучающей выборке и сделаем предсказание на валидационной

In [11]:
model1 = LinearRegression().fit(features_train1, target_train1)
predicted_valid1 = model1.predict(features_valid1)

In [12]:
model2 = LinearRegression().fit(features_train2, target_train2)
predicted_valid2 = model2.predict(features_valid2)

In [13]:
model3 = LinearRegression().fit(features_train3, target_train3)
predicted_valid3 = model3.predict(features_valid3)

### Среднее значение предсказанного сырья и RMSE
Посмотрим средний запас предсказанного сырья и RMSE модели

In [14]:
rmse1 = mean_squared_error(target_valid1, predicted_valid1)** 0.5
print("Средний запас сырья 1 региона (тыс. баррелей) =", predicted_valid1.mean())
print("RMSE1 =", rmse1)

Средний запас сырья 1 региона (тыс. баррелей) = 92.59256778438038
RMSE1 = 37.5794217150813


In [15]:
rmse2 = mean_squared_error(target_valid2, predicted_valid2)** 0.5
print("Средний запас сырья 2 региона (тыс. баррелей) =", predicted_valid2.mean())
print("RMSE1 =", rmse2)

Средний запас сырья 2 региона (тыс. баррелей) = 68.728546895446
RMSE1 = 0.893099286775616


In [16]:
rmse3 = mean_squared_error(target_valid3, predicted_valid3)** 0.5
print("Средний запас сырья 3 региона (тыс. баррелей) =", predicted_valid3.mean())
print("RMSE1 =", rmse3)

Средний запас сырья 3 региона (тыс. баррелей) = 94.96504596800489
RMSE1 = 40.02970873393434


Самый большой запас сырья предсказывает 3 модель, но у нее же и самый большой корень из среднеквадратической ошибки, самая лучшая модель по RMSE - вторая.

## Подготовка к расчёту прибыли

Данные для подсчета прибыли

In [17]:
BUDGET = 10000 #бюджет в миллионах рублей
BARREL_REVENUE = 0.45 #доход с тысячи баррелей в миллионах рублей

In [18]:
VALUE_FOR_REVENUE = BUDGET / 200 / BARREL_REVENUE
VALUE_FOR_REVENUE

111.11111111111111

Для того, чтобы разработка новой скважины была безубыточной, необходимо добывать больше 111,1 тысячи баррелей сырья.

Посчитаем разницу среднего запаса сырья по всем скважинам региона и необходимого объема для безубыточной работы.

In [19]:
mean_value1 = df1['product'].mean()
(mean_value1 - VALUE_FOR_REVENUE)

-18.6111111111111

In [20]:
mean_value2 = df2['product'].mean()
(mean_value2 - VALUE_FOR_REVENUE)

-42.2861111111111

In [21]:
mean_value3 = df3['product'].mean()
(mean_value3 - VALUE_FOR_REVENUE)

-16.11111111111107

Во всех регионах среднее значение сырья по всем скважинам недостаточно для безубыточной разработки. Следовательно необходимо использовать предсказания моделей для определения наиболее эффективных скважин.

## Расчёт прибыли и рисков 

### Расчёт прибыли
Напишем функцию __profit__ для определения прибыли для региона, которая выбирает 200 скважин с максимальными значениями предсказаний, суммирует целевое значение объёма сырья, соответствующее этим предсказаниям, и возвращает прибыль в миллионах рублей.

In [22]:
def profit(target, predictions):
    pred_choice = pd.Series(predictions).sort_values(ascending=False).head(200) 
    target_choice = target.reset_index(drop=True)[pred_choice.index] 
    return target_choice.sum() *  BARREL_REVENUE - BUDGET

Посчитаем прибыль для первого региона: для этого применим функцию __revenue__ к валидационной выборке целевого признака и предсказаний.

In [23]:
profit(target_valid1, predicted_valid1)

3320.826043139852

Посчитаем прибыль для второго региона.

In [24]:
profit(target_valid2, predicted_valid2)

2415.086696681512

Посчитаем прибыль для третьего региона.

In [25]:
profit(target_valid3, predicted_valid3)

2710.3499635998323

В первом регионе прибыль - 3320 миллионов рублей, для вторго региона модель предсказывает прибыль в 2415 миллионов рублей, для третьего региона прибыль - 2710 миллионов рублей.

### Расчёт рисков

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли, посчитаем среднее значение прибыли, 95%-й доверительный интервал и риск убытков (сделаем счетчик выборок с отрицательной прибылью, потом разделим его на 10, чтоб получить значение в процентах).

__Для первого региона__

In [26]:
state = np.random.RandomState(12345) 
values1 = []
loss1 = 0

for i in range(1000):
    target_subsample = target_valid1.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
    probs_subsample = predicted_valid1[target_subsample.index]
    values1.append(profit(target_subsample, probs_subsample))
    if profit(target_subsample, probs_subsample) < 0:
        loss1 += 1

In [27]:
values1 = pd.Series(values1)

profit1 = values1.mean()

confidence_interval1 = st.t.interval(0.95, len(values1)-1, loc = profit1, scale = values1.sem())

(profit1, confidence_interval1, loss1 / 10)

(396.1649848023714, (379.6203151479728, 412.70965445677), 6.9)

__Для второго региона__

In [28]:
state = np.random.RandomState(12345) 
values2 = []
loss2 = 0 

for i in range(1000):
    target_subsample = target_valid2.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
    probs_subsample = predicted_valid2[target_subsample.index]
    values2.append(profit(target_subsample, probs_subsample))
    if profit(target_subsample, probs_subsample) < 0:
        loss2 += 1

In [29]:
values2 = pd.Series(values2)

profit2 = values2.mean()

confidence_interval2 = st.t.interval(0.95, len(values2)-1, loc = profit2, scale = values2.sem())

(profit2, confidence_interval2, loss2 / 10)

(456.04510578666117, (443.14724866390094, 468.9429629094214), 1.5)

__Для третьего региона__

In [30]:
state = np.random.RandomState(12345) 
values3 = []
loss3 = 0  

for i in range(1000):
    target_subsample = target_valid3.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
    probs_subsample = predicted_valid3[target_subsample.index]
    values3.append(profit(target_subsample, probs_subsample))
    if profit(target_subsample, probs_subsample) < 0:
        loss3 += 1

In [31]:
values3 = pd.Series(values3)

profit3 = values3.mean()

confidence_interval3 = st.t.interval(0.95, len(values3)-1, loc = profit3, scale = values3.sem())

(profit3, confidence_interval3, loss3 / 10)

(404.4038665683571, (387.4457974712807, 421.3619356654335), 7.6)

Посчитаем 95% доверительный интервал методом квантилей.

In [32]:
lower1 = values1.quantile(0.025)
upper1 = values1.quantile(0.975)
(lower1, upper1)

(-111.21554589049532, 909.766941553422)

In [33]:
lower2 = values2.quantile(0.025)
upper2 = values2.quantile(0.975)
(lower2, upper2)

(33.82050939898718, 852.2894538660361)

In [34]:
lower3 = values3.quantile(0.025)
upper3 = values3.quantile(0.975)
(lower3, upper3)

(-163.350413395599, 950.3595749238)

Выведем полученные значения для всех регионов в сводную таблицу.

In [35]:
pivot = pd.DataFrame({'profit': [profit1, profit2, profit3], 
                      'confidence_interval': [(lower1, upper1), (lower2, upper2), (lower3, upper3)], 
                      'loss': [loss1 / 10, loss2 / 10, loss3 / 10]})

pivot.index = ['Region_1', 'Region_2', 'Region_3']

pivot

,profit,confidence_interval,loss
Region_1,396.164985,"(-111.21554589049532, 909.766941553422)",6.9
Region_2,456.045106,"(33.82050939898718, 852.2894538660361)",1.5
Region_3,404.403867,"(-163.350413395599, 950.3595749238)",7.6


Лишь у одного региона получился риск убытков меньше 2,5%, у него же и самая высокая прибыль.

## Выводы

В данном проекте были построены модели для предсказания объемов добычи нефти для трех регионов по характеристикам скважин.

Если смотреть прибыль регионов по 200 скважинам, соответствующим лучшим по предсказанниям скважинам из валидационной выборки (25000 скважин), то получается:
- Первый регион: 3320 миллионов рублей;
- Второй регион: 2415 миллионов рублей;
- Третий регион: 2710 миллионов рублей.

Но поскольку исследование скважин очень затратное, и бюджет рассчитан на исследование только 500 скважин в регионе, то была применина процедура Bootstrap для исследования 1000 выборок по 500 скважин из валидационной выборки. Было получено распределение прибылей для 1000 вариантов, и найдены средняя прибыль для региона, 95%-й доверительный интервал и риск убытков. 

Средняя прибыль для регионов после применения процедуры Bootstrap:
- Первый регион: 396 миллионов рублей;
- Второй регион: 456 миллионов рублей;
- Третий регион: 404 миллиона рублей.
 Очевидно что если проводить исследование только 500 скважин, то прибыль падает относительно прибыли, если исследовать больше скважин. Возможно, компании «ГлавРосГосНефть» стоит предоставить данные о стоимости исследования одной скважины и рассчитать более оптимальное число скважин для исследования.
  
  По полученным данным только у 2 региона оказались риски убытков меньше 2,5%, и также самая большая прибыльность, следовательно его и стоит рекомендовать для выбора компании «ГлавРосГосНефть».